# RRAP-IS Dataset sync Demo Notebook

> A tutorial of RRAP uploading and downloading dataset using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

## About

This notebook is a demonstration of downloading and uploading dataset to RRAP data repository. 

### Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [1]:
import requests
import os
import sys
import json
from bs4 import BeautifulSoup
from IPython.display import IFrame, display, HTML, JSON, Markdown, Image
from mdsisclienttools.auth.TokenManager import DeviceFlowManager
import mdsisclienttools.datastore.ReadWriteHelper as IOHelper
import numpy as np
import pandas as pd

from cloudpathlib import S3Client
import cloudpathlib
import warnings
warnings.filterwarnings(action='once')

### Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [2]:
data_store = "https://data.testing.rrap-is.com"
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server, 'data_store': data_store}#, 'garbage': garbage}
display(f'Checking base urls')

for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed
Testing - https://data.testing.rrap-is.com - Passed


## Authentication

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

[Return to Top](#toc)

In [3]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Found tokens valid, using.



## Endpoint Documentation
Endpoint documentation can be found by appending either `/docs` or `/redoc` on the end a base URL.

For example:
<ul>
  <li><a href="https://data-api.testing.rrap-is.com/redoc" target="_blank">Redoc FastAPI</a></li>
  <li><a href="https://data-api.testing.rrap-is.com/docs" target="_blank">Docs FastAPI</a> </li>
</ul>

Then select from the menu an endpoint function call e.g. `/register/mint-dataset`

Then append the function call onto the base url e.g. `https://data-api.testing.rrap-is.com/register/mint-dataset`

[Return to Top](#toc)

## Demonstration

Initially we need to have a registered s3 folder, once this is done we can upload and download datasets to it.

### Register a dataset
see [Rigistering and uploading a dataset](https://gbrrestoration.github.io/rrap-mds-knowledge-hub/information-system/data-store/registering-and-uploading-a-dataset.html)


In [4]:
# Using RRAP-IS python packages
auth = token_manager.get_auth
postfix = "/register/mint-dataset"
payload =  {
  "author": {
    "name": "Andrew Freebairn",
    "email": "andrew.freebairn@csiro.au",
    "orcid": "https://orcid.org/0000-0001-9429-6559",
    "organisation": {
      "name": "CSIRO",
      "ror": "https://ror.org/03qn8fb07"
    }
  },
  "dataset_info": {
    "name": "MVP Demo Dataset",
    "description": "For demonstration purposes",
    "publisher": {
      "name": "Andrew",
      "ror": "https://ror.org/057xz1h85"
    },
    "created_date": "2022-08-05",
    "published_date": "2022-08-05",
    "license": "https://creativecommons.org/licenses/by/4.0/",
    "keywords": [
      "keyword1"
    ],
    "version": "0.0.1"
  }
}
endpoint = data_api + postfix 

response = requests.post(endpoint, json=payload, auth=auth())

print(json.dumps(response.json(), indent=2))

{
  "status": {
    "success": true,
    "details": "Successfully seeded location - see location details."
  },
  "handle": "10378.1/1688092",
  "s3_location": {
    "bucket_name": "dev-rrap-storage-bucket",
    "path": "datasets/10378-1-1688092/",
    "s3_uri": "s3://dev-rrap-storage-bucket/datasets/10378-1-1688092/"
  }
}


#### Upload data associated with registered data

In [ ]:
auth = token_manager.get_auth
IOHelper.DEFAULT_DATA_STORE_ENDPOINT = data_store
IOHelper.upload("10378.1/1688081", auth, "./data")

#### Download data from a data registry

In [ ]:
auth = token_manager.get_auth
IOHelper.DEFAULT_DATA_STORE_ENDPOINT = data_store
IOHelper.download('./data', "10378.1/1688081", auth)